In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
# key => 
# 산업안전보건법 + "1"
# 건설기술진흥법 + "2"
# 위험물안전관리법 + '3'
# 전기안전관리법 + "4"
# 화학물질관리법 + "5"

In [2]:
df_industrial = pd.read_excel("산업안전보건법.xlsx")
df_erection = pd.read_excel("건설기술진흥법.xlsx")
df_danger = pd.read_excel("위험물안전관리법.xlsx")
df_electric = pd.read_excel("전기안전관리법.xlsx")
df_chemistry = pd.read_excel("화학물질관리법.xlsx")
df_info = pd.DataFrame()
df_info = df_info.append(df_industrial,ignore_index = True)
df_info = df_info.append(df_erection,ignore_index = True)
df_info = df_info.append(df_danger,ignore_index = True)
df_info = df_info.append(df_electric,ignore_index = True)
df_info = df_info.append(df_chemistry,ignore_index = True)
df_info

,키,조내용
0,10011,제1조(목적) 이 법은 산업 안전 및 보건에 관한 기준을 확립하고 그 책임의 소재를...
1,20011,제2조(정의) 이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2020.5.2...
2,30011,"제3조(적용 범위) 이 법은 모든 사업에 적용한다. 다만, 유해ㆍ위험의 정도, 사업..."
3,40011,제4조(정부의 책무) 정부는 이 법의 목적을 달성하기 위하여 다음 각 호의 사항을 ...
4,50011,제5조(사업주 등의 의무) 사업주(제77조에 따른 특수형태근로종사자로부터 노무를 제...
...,...,...
452,600015,제60조(벌칙) 제43조제2항에 따라 즉시 신고를 하지 아니한 자는 2년 이하의 징...
453,610015,제61조(벌칙) 다음 각 호의 어느 하나에 해당하는 자는 1년 이하의 징역 또는 3...
454,620015,제62조(벌칙) 다음 각 호의 어느 하나에 해당하는 자는 6개월 이하의 징역 또는 ...
455,630015,"제63조(양벌규정) 법인의 대표자나 법인 또는 개인의 대리인, 사용인, 그 밖의 종..."


In [3]:
#df_info.to_excel('전체법.xlsx',index = False)

In [4]:
# 불용어 리스트
# 불용어, 불필요 단어 제거
stop_words_df = pd.read_excel("stopwords.xlsx")
stop_words_df

,불용어
0,아
1,휴
2,아이구
3,아이쿠
4,아이고
...,...
695,필요한
696,등을
697,해당하는
698,자가


In [5]:
posts = df_info.get("조내용")
posts_key = df_info.get("키")
key = [i for i in posts_key]
posts

0      제1조(목적) 이 법은 산업 안전 및 보건에 관한 기준을 확립하고 그 책임의 소재를...
1      제2조(정의) 이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2020.5.2...
2      제3조(적용 범위) 이 법은 모든 사업에 적용한다. 다만, 유해ㆍ위험의 정도, 사업...
3      제4조(정부의 책무) 정부는 이 법의 목적을 달성하기 위하여 다음 각 호의 사항을 ...
4      제5조(사업주 등의 의무) 사업주(제77조에 따른 특수형태근로종사자로부터 노무를 제...
                             ...                        
452    제60조(벌칙) 제43조제2항에 따라 즉시 신고를 하지 아니한 자는 2년 이하의 징...
453    제61조(벌칙) 다음 각 호의 어느 하나에 해당하는 자는 1년 이하의 징역 또는 3...
454    제62조(벌칙) 다음 각 호의 어느 하나에 해당하는 자는 6개월 이하의 징역 또는 ...
455    제63조(양벌규정) 법인의 대표자나 법인 또는 개인의 대리인, 사용인, 그 밖의 종...
456    제64조(과태료) 다음 각 호의 어느 하나에 해당하는 자에게는 1천만원 이하의 과태...
Name: 조내용, Length: 457, dtype: object

In [6]:
def get_key(**key) :
    dic = {}
    for k, y in key.items():
        if k[-1] == "1" :
            s = "OSH"+k[0:(len(k)-4)]
        elif k[-1]=="2":
            s = "CTP"+k[0:(len(k)-4)]
        elif k[-1]=="3":
            s = "HSM"+k[0:(len(k)-4)]
        elif k[-1]=="4":
            s = "ESM"+k[0:(len(k)-4)]
        elif k[-1]=="5":
            s = "CMM"+k[0:(len(k)-4)]     
        if k[-3] != "0":
            s += "의 "+k[-3]   
        dic[s] = y
    return dic   

In [7]:
def get_law(law):
    if law == 'OSH':
        num = '1'
    elif law == 'CTP':
        num = '2'
    elif law == 'HSM':
        num = '3'
    elif law == 'ESM':
        num = '4'
    elif law == 'CMM':
        num = '5'
    return num

In [8]:
from konlpy.tag import Okt
okt = Okt()

oo = okt.pos(posts[0],
        norm=True,   # 정규화(normalization)
        stem=True    # 어간추출(stemming)
        )
print(oo)

[('제', 'Noun'), ('1조', 'Number'), ('(', 'Punctuation'), ('목적', 'Noun'), (')', 'Punctuation'), ('이', 'Noun'), ('법', 'Noun'), ('은', 'Josa'), ('산업', 'Noun'), ('안전', 'Noun'), ('및', 'Noun'), ('보건', 'Noun'), ('에', 'Josa'), ('관', 'Noun'), ('한', 'Josa'), ('기준', 'Noun'), ('을', 'Josa'), ('확립', 'Noun'), ('하고', 'Josa'), ('그', 'Noun'), ('책임', 'Noun'), ('의', 'Josa'), ('소재', 'Noun'), ('를', 'Josa'), ('명확하다', 'Adjective'), ('하다', 'Verb'), ('산업', 'Noun'), ('재해', 'Noun'), ('를', 'Josa'), ('예방', 'Noun'), ('하고', 'Josa'), ('쾌적하다', 'Adjective'), ('작업', 'Noun'), ('환경', 'Noun'), ('을', 'Josa'), ('조', 'Modifier'), ('성함', 'Noun'), ('으로써', 'Josa'), ('노무', 'Noun'), ('를', 'Josa'), ('제공', 'Noun'), ('하다', 'Verb'), ('사람', 'Noun'), ('의', 'Josa'), ('안전', 'Noun'), ('및', 'Noun'), ('보건', 'Noun'), ('을', 'Josa'), ('유지', 'Noun'), ('ㆍ', 'Foreign'), ('증진', 'Noun'), ('함', 'Noun'), ('을', 'Josa'), ('목적', 'Noun'), ('으로', 'Josa'), ('하다', 'Verb'), ('.', 'Punctuation'), ('<', 'Punctuation'), ('개정', 'Noun'), ('2020.5', 'Number'), ('.', '

In [9]:
def tokenizer_1(raw_texts, pos=["Noun","Alpha","Verb","Number"], stopword=list(stop_words_df.get("불용어"))):
    p = okt.pos(raw_texts, 
            norm=True,   # 정규화(normalization)
            stem=True    # 어간추출(stemming)
            )
    o = [word for word, tag in p if len(word) > 1 and tag in pos and word[0] not in stopword]
    return(o)

tokenizer_1(posts[80])


['81조',
 '기계',
 '기구',
 '대통령령',
 '기계',
 '기구',
 '설비',
 '건축물',
 '타인',
 '대여',
 '거나',
 '대여받다',
 '안전',
 '보건']

In [10]:
from konlpy.tag import Mecab
tagger = Mecab()
# from eunjeon import Mecab
# tagger = Mecab()
def tokenizer_2(raw_texts, pos=["Noun","Alpha","Verb","Number"], stop_words=list(stop_words_df.get("불용어"))):
    nouns = []
  
    for noun in tagger.nouns(raw_texts):
        if noun not in stop_words:
            nouns.append(noun)
    return nouns

tokenizer_2(posts[80])

['기계', '대여자', '조치', '기계', '건축물', '대여', '필요', '안전', '조치', '보건', '조치']

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorize = TfidfVectorizer(
    tokenizer=tokenizer_1, # 문장에 대한 tokenizer (위에 정의한 함수 이용)
    min_df=1,            # 단어가 출현하는 최소 문서의 개수
    sublinear_tf=True    # tf값에 1+log(tf)를 적용하여 tf값이 무한정 커지는 것을 막음
)

X = vectorize.fit_transform(posts)
X.toarray() 

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [12]:
# 모든 조에 해당하는 euclidean distance 를 계산
euclidean_result = []
for x in range(len(df_info)):
    tmp = []
    for y in range(len(df_info)):
        tmp.append(euclidean_distances(X[x], X[y])[0][0])
    euclidean_result.append(tmp)
# euclidean_result

In [13]:
# 모든 조에 해당하는 euclidean distance 결과 값을
# DataFrame 형태로 변환
euclidean_result_df = pd.DataFrame(euclidean_result)
euclidean_result_df

,0,1,2,3,4,5,6,7,8,9,...,447,448,449,450,451,452,453,454,455,456
0,0.000000,1.287018,1.414214,1.165998,1.254766,1.329572,1.345013,1.365101,1.306567,1.338770,...,1.352337,1.372577,1.414214,1.404163,1.386000,1.414214,1.409765,1.409174,1.414214,1.409252
1,1.287018,0.000000,1.297864,1.330648,1.230558,1.270694,1.315033,1.335517,1.343985,1.210560,...,1.371830,1.376658,1.401742,1.382693,1.379936,1.412670,1.381800,1.391088,1.384141,1.374472
2,1.414214,1.297864,0.000000,1.414214,1.377286,1.382473,1.410350,1.400083,1.407667,1.381674,...,1.403273,1.286184,1.414214,1.394130,1.397979,1.414214,1.399753,1.404557,1.397930,1.393891
3,1.165998,1.330648,1.414214,0.000000,1.310313,1.244209,1.333259,1.291583,1.247382,1.355435,...,1.252486,1.300174,1.414214,1.400123,1.375494,1.410904,1.396837,1.403805,1.391467,1.397268
4,1.254766,1.230558,1.377286,1.310313,0.000000,1.174423,1.358981,1.350860,1.334742,1.316628,...,1.376101,1.387900,1.414214,1.358628,1.346798,1.408668,1.361305,1.368473,1.396279,1.392412
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
452,1.414214,1.412670,1.414214,1.410904,1.408668,1.403512,1.408280,1.408634,1.407063,1.408073,...,1.404831,1.374507,1.311241,1.296584,1.357984,0.000000,1.288660,1.328792,1.414214,1.368187
453,1.409765,1.381800,1.399753,1.396837,1.361305,1.391531,1.365614,1.384728,1.406121,1.397338,...,1.400022,1.389328,1.343239,1.028503,1.226607,1.288660,0.000000,1.049590,1.403860,1.065491
454,1.409174,1.391088,1.404557,1.403805,1.368473,1.406960,1.391645,1.387743,1.409365,1.403171,...,1.403820,1.386849,1.373870,1.128706,1.212853,1.328792,1.049590,0.000000,1.363034,1.211996
455,1.414214,1.384141,1.397930,1.391467,1.396279,1.393293,1.411458,1.380973,1.414214,1.409965,...,1.403364,1.393448,1.350949,1.371245,1.344552,1.414214,1.403860,1.363034,0.000000,1.382751


In [14]:
law_jo = 'OSH24'
        
if law_jo[-3] == '의':
    if len(law_jo) == 7 :
        jo_key = law_jo[3] + '0' + law_jo[-1] + '1'+ get_law(law_jo[0:3])
    elif len(law_jo) == 8 :
        jo_key = law_jo[3:5] + '0' + law_jo[-1] + '1'+ get_law(law_jo[0:3])
    elif len(law_jo) == 9 :
        jo_key = law_jo[3:6] + '0' + law_jo[-1] + '1'+ get_law(law_jo[0:3])
else:    
    jo_key = law_jo[3:] + '001' + get_law(law_jo[0:3])

In [15]:
# euclidean distance 값으로 
# 8조와 가장 비슷한 조를 찾기위한 과정
# jo_key = 240011
jo = key.index(int(jo_key))
y_euclidean = euclidean_result_df[jo]
dic = {}
for i in range(len(y_euclidean)):
    dic[str(posts_key[i])] = y_euclidean[i] 
dic = get_key(**dic)
#8조와 비슷한 조 찾기 위함
# euclidean distance 를 dic에 씌워 sorting 함(1부터 10위까지)
sorted(dic.items(), key=lambda x: x[1])[0:15]

[('OSH24', 0.0),
 ('OSH75', 0.9936866276722346),
 ('OSH26', 1.0506381267109706),
 ('OSH25', 1.0809230422198084),
 ('OSH49', 1.1415892344218668),
 ('OSH44', 1.1557100725075922),
 ('OSH116', 1.1579327620405564),
 ('OSH22', 1.1609106485376792),
 ('CMM7', 1.1783897533844376),
 ('CTP84', 1.1825741507664376),
 ('OSH18', 1.2039747191785233),
 ('OSH47', 1.210152673292496),
 ('OSH32', 1.2186579340619614),
 ('OSH142', 1.2213242655582994),
 ('OSH35', 1.2213332526696865)]

In [16]:
cosine_result = []
for x in range(len(df_info)):
    tmp = []
    for y in range(len(df_info)):
        tmp.append(cosine_similarity(X[x], X[y])[0][0])
    cosine_result.append(tmp)

In [17]:
# 모든 조에 해당하는 cosine similarity 결과 값을
# DataFrame 형태로 변환
cosine_result_df = pd.DataFrame(cosine_result)
cosine_result_df

,0,1,2,3,4,5,6,7,8,9,...,447,448,449,450,451,452,453,454,455,456
0,1.000000,0.171792,0.000000,0.320224,0.212782,0.116119,0.095470,0.068250,0.146441,0.103847,...,0.085593,0.058016,0.000000,0.014163,0.039502,0.000000,0.006282,0.007115,0.000000,0.007004
1,0.171792,1.000000,0.157774,0.114688,0.242864,0.192669,0.135344,0.108197,0.096852,0.267272,...,0.059041,0.052407,0.017560,0.044080,0.047889,0.002181,0.045315,0.032437,0.042077,0.055414
2,0.000000,0.157774,1.000000,0.000000,0.051542,0.044384,0.005457,0.019884,0.009236,0.045489,...,0.015413,0.172865,0.000000,0.028201,0.022828,0.000000,0.020346,0.013610,0.022896,0.028534
3,0.320224,0.114688,0.000000,1.000000,0.141540,0.225972,0.111210,0.165906,0.222020,0.081398,...,0.215639,0.154774,0.000000,0.019828,0.054009,0.004676,0.024424,0.014666,0.031910,0.023821
4,0.212782,0.242864,0.051542,0.141540,1.000000,0.310365,0.076585,0.087589,0.109232,0.133245,...,0.053173,0.036866,0.000000,0.077065,0.093068,0.007827,0.073424,0.063641,0.025203,0.030594
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
452,0.000000,0.002181,0.000000,0.004676,0.007827,0.015077,0.008373,0.007875,0.010086,0.008666,...,0.013224,0.055365,0.140323,0.159435,0.077940,1.000000,0.169678,0.117156,0.000000,0.064032
453,0.006282,0.045315,0.020346,0.024424,0.073424,0.031821,0.067550,0.041265,0.011412,0.023723,...,0.019969,0.034884,0.097854,0.471091,0.247718,0.169678,1.000000,0.449180,0.014588,0.432364
454,0.007115,0.032437,0.013610,0.014666,0.063641,0.010232,0.031663,0.037084,0.006845,0.015555,...,0.014645,0.038325,0.056240,0.363011,0.264494,0.117156,0.449180,1.000000,0.071069,0.265533
455,0.000000,0.042077,0.022896,0.031910,0.025203,0.029367,0.003893,0.046457,0.000000,0.005999,...,0.015284,0.029151,0.087468,0.059843,0.096089,0.000000,0.014588,0.071069,1.000000,0.044000


In [18]:
# cosine similarity 값으로 
# 8조와 가장 비슷한 조를 찾기위한 과정
y_cosine = cosine_result_df[jo]
dic ={}
for i in range(len(y_cosine)):
    dic[str(posts_key[i])] = y_cosine[i]
dic = get_key(**dic)
#8조와 비슷한 조 찾기 위함
# euclidean distance 를 dic에 씌워 sorting 함(1부터 10위까지)
sorted(dic.items(), key=lambda x: x[1], reverse = True)[0:10]

[('OSH24', 1.0),
 ('OSH75', 0.5062934429926911),
 ('OSH26', 0.4480797633506314),
 ('OSH25', 0.41580268839913725),
 ('OSH49', 0.348387009926048),
 ('OSH44', 0.3321671141522481),
 ('OSH116', 0.329595859296564),
 ('OSH22', 0.32614323305591253),
 ('CMM7', 0.30569879455928217),
 ('CTP84', 0.3007591889695195)]

In [19]:
import openpyxl
# 워크북 열기
wb = openpyxl.load_workbook('TF-IDF결과.xlsx')
# 시트 열기(활성화)
sheet = wb['안전보건법령 간']

# 제시된 틀 만들기
sheet['A1'] = '안전보건법령 간 TF-IDF 기준'
for i in range(10) :
    sheet.cell(row=2, column=i+2).value = i+1
for i in range(0,141) :
    s = "OSH"+str(i+1)
    sheet.cell(row=i+3, column=1).value = s

# 상위 유사조항 10개   
for j in range(0,141) :
    y_cosine = cosine_result_df[j]
    dic ={}
    for i in range(len(y_cosine)):
        dic[str(posts_key[i])] = y_cosine[i]
    dic = get_key(**dic)
    a = sorted(dic.items(), key=lambda x: x[1], reverse = True)[1:11]
    for i in range(10) :
        sheet.cell(row=j+3, column=i+2).value = str(a[i][0])

# 저장
wb.save('TF-IDF결과.xlsx')

In [20]:
from sklearn.feature_extraction.text import CountVectorizer
countv = CountVectorizer(stop_words = list(stop_words_df.get("불용어")))
sp_mat = countv.fit_transform(posts)
# 희소행렬을 np array로
sp_mat.toarray()

/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['가서', '같은', '것과', '결과에', '결론을', '관계가', '관련이', '그런', '그럼에도', '그렇게', '그에', '그치지', '김에', '까닭에', '낫다', '년도', '논하지', '누가', '다시', '달려', '대로', '대해', '되는', '되다', '되어', '들면', '들자면', '듯하다', '따르는', '따름이다', '따지지', '때가', '만은', '만이', '많은', '말하면', '말할것도', '몰라도', '몰랏다', '못하다', '못하다하기보다는', '미치다', '바꾸어서', '바꿔', '방면으로', '보면', '보아', '부류의', '비길수', '비추어', '뿐만', '사람들', '상대적으로', '생각이다', '서술한바와같이', '쓰여', '아니다', '아니라', '안다', '안된다', '않고', '않기', '않는다면', '않다', '않다면', '않도록', '않으면', '알겠는가', '어쩔수', '없고', '없다', '예를', '외에', '요만한', '우에', '위에서', '이렇게', '이로', '이르다', '이와', '이유는', '인하여', '임에', '점에서', '정도에', '정도의', '종합한것과같이', '주저하지', '줄은', '지경이다', '틀림없다', '편이', '하고', '하기', '하기만', '하는', '하는것만', '하는것이', '하면', '하지', '한하다', '할수록', '함으로써', '해도', '해서는', '형식으로', '힘이'] not in stop_words.
  warnings.warn('Your stop_words may be inconsis

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [21]:
vectorize = CountVectorizer(
    tokenizer=tokenizer_2, # 문장에 대한 tokenizer (위에 정의한 함수 이용)
    min_df=1,            # 단어가 출현하는 최소 문서의 개수
)

X = vectorize.fit_transform(posts)
X.toarray() 

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [22]:
# 모든 조에 해당하는 euclidean distance 를 계산
euclidean_result = []
for x in range(len(df_info)):
    tmp = []
    for y in range(len(df_info)):
        tmp.append(euclidean_distances(X[x], X[y])[0][0])
    euclidean_result.append(tmp)
# euclidean_result

In [23]:
# 모든 조에 해당하는 euclidean distance 결과 값을
# DataFrame 형태로 변환
euclidean_result_df = pd.DataFrame(euclidean_result)
euclidean_result_df

,0,1,2,3,4,5,6,7,8,9,...,447,448,449,450,451,452,453,454,455,456
0,0.000000,25.019992,8.717798,13.820275,11.135529,7.071068,9.433981,24.351591,12.083046,16.155494,...,8.246211,6.708204,6.480741,19.131126,22.068076,6.082763,13.416408,11.045361,10.000000,30.919250
1,25.019992,0.000000,24.331050,28.266588,24.083189,24.819347,26.172505,33.511192,27.129320,27.147744,...,25.961510,25.670995,25.845696,31.144823,33.120990,25.748786,28.248894,27.202941,26.419690,39.115214
2,8.717798,24.331050,0.000000,17.175564,12.409674,8.602325,10.908712,24.596748,13.638182,16.522712,...,9.055385,7.810250,8.124038,19.646883,22.649503,7.681146,14.282857,12.000000,11.224972,31.112698
3,13.820275,28.266588,17.175564,0.000000,16.401219,15.652476,17.435596,26.495283,17.691806,21.725561,...,16.643317,16.613248,16.462078,24.062419,26.115130,16.309506,20.074860,18.734994,18.138357,34.278273
4,11.135529,24.083189,12.409674,16.401219,0.000000,9.899495,13.674794,24.879711,14.832397,17.691806,...,12.727922,12.767145,12.328828,21.213203,23.558438,12.124356,16.431677,15.033296,14.628739,32.372828
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
452,6.082763,25.748786,7.681146,16.309506,12.124356,6.855655,8.717798,24.248711,12.288206,15.684387,...,7.937254,5.291503,3.316625,18.466185,21.633308,0.000000,12.041595,9.539392,9.110434,30.215890
453,13.416408,28.248894,14.282857,20.074860,16.431677,13.638182,14.525839,26.814175,16.733201,19.519221,...,14.491377,13.304135,12.247449,14.491377,19.621417,12.041595,0.000000,9.695360,15.231546,23.409400
454,11.045361,27.202941,12.000000,18.734994,15.033296,11.575837,12.449900,25.826343,15.427249,18.248288,...,12.165525,10.723805,9.899495,15.684387,19.467922,9.539392,9.695360,0.000000,12.727922,27.349589
455,10.000000,26.419690,11.224972,18.138357,14.628739,10.488088,11.789826,25.514702,14.560220,17.748239,...,11.045361,9.433981,9.486833,19.748418,22.516660,9.110434,15.231546,12.727922,0.000000,31.240999


In [24]:
# euclidean distance 값으로 
# 8조와 가장 비슷한 조를 찾기위한 과정
y_euclidean = euclidean_result_df[jo]
dic = {}
for i in range(len(y_euclidean)):
    dic[str(posts_key[i])] = y_euclidean[i]
dic = get_key(**dic)
#8조와 비슷한 조 찾기 위함
# euclidean distance 를 dic에 씌워 sorting 함(1부터 10위까지)
sorted(dic.items(), key=lambda x: x[1])[0:10]

[('OSH24', 0.0),
 ('OSH25', 18.275666882497067),
 ('OSH49', 18.439088914585774),
 ('OSH32', 19.05255888325765),
 ('OSH35', 19.6468827043885),
 ('OSH26', 19.72308292331602),
 ('OSH4', 19.8997487421324),
 ('OSH19', 19.8997487421324),
 ('OSH31', 20.71231517720798),
 ('OSH50', 20.784609690826528)]

In [25]:
import openpyxl
# 워크북 열기
wb = openpyxl.load_workbook('count결과(Eu).xlsx')
# 시트 열기(활성화)
sheet = wb['안전보건법령 간']

# 제시된 틀 만들기
sheet['A1'] = '안전보건법령 간 count결과(Eu) 기준'
for i in range(10) :
    sheet.cell(row=2, column=i+2).value = i+1
for i in range(0,141) :
    s = "OSH"+str(i+1)
    sheet.cell(row=i+3, column=1).value = s

# 상위 유사조항 10개   
for j in range(0,141) :
    y_euclidean = euclidean_result_df[j]
    dic ={}
    for i in range(len(y_euclidean)):
        dic[str(posts_key[i])] = y_euclidean[i]
    dic = get_key(**dic)
    a = sorted(dic.items(), key=lambda x: x[1])[1:11]
    for i in range(10) :
        sheet.cell(row=j+3, column=i+2).value = str(a[i][0])

# 저장
wb.save('count결과(Eu).xlsx')

In [26]:
cosine_result = []
for x in range(len(df_info)):
    tmp = []
    for y in range(len(df_info)):
        tmp.append(cosine_similarity(X[x], X[y])[0][0])
    cosine_result.append(tmp)

In [27]:
# 모든 조에 해당하는 cosine similarity 결과 값을
# DataFrame 형태로 변환
cosine_result_df = pd.DataFrame(cosine_result)
cosine_result_df

,0,1,2,3,4,5,6,7,8,9,...,447,448,449,450,451,452,453,454,455,456
0,1.000000,0.203341,0.052870,0.572892,0.383571,0.221470,0.047378,0.063819,0.178279,0.061602,...,0.158610,0.043033,0.046676,0.051508,0.061877,0.000000,0.045787,0.019540,0.000000,0.018881
1,0.203341,1.000000,0.306394,0.138335,0.354230,0.238037,0.081888,0.090838,0.095572,0.200419,...,0.086005,0.061253,0.009491,0.033516,0.026962,0.000000,0.021724,0.019867,0.064122,0.037114
2,0.052870,0.306394,1.000000,0.076669,0.250958,0.180656,0.000000,0.085407,0.046271,0.105523,...,0.226415,0.184289,0.066630,0.058822,0.044164,0.086874,0.043574,0.041841,0.000000,0.044922
3,0.572892,0.138335,0.076669,1.000000,0.350218,0.265085,0.083972,0.179949,0.229803,0.063524,...,0.153337,0.027735,0.045124,0.049796,0.074062,0.039223,0.039346,0.018891,0.014517,0.020282
4,0.383571,0.354230,0.250958,0.350218,1.000000,0.573409,0.122666,0.185887,0.230792,0.202025,...,0.205329,0.018570,0.100707,0.097797,0.122061,0.105045,0.111961,0.067456,0.019439,0.043454
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
452,0.000000,0.000000,0.086874,0.039223,0.105045,0.000000,0.000000,0.026216,0.000000,0.080978,...,0.000000,0.070711,0.613572,0.152346,0.116199,1.000000,0.300942,0.256865,0.000000,0.165468
453,0.045787,0.021724,0.043574,0.039346,0.111961,0.065188,0.068333,0.036160,0.066787,0.055847,...,0.010893,0.017733,0.250046,0.632521,0.451683,0.300942,1.000000,0.652230,0.000000,0.708041
454,0.019540,0.019867,0.041841,0.018891,0.067456,0.000000,0.049992,0.025252,0.000000,0.026000,...,0.013947,0.022704,0.197006,0.543502,0.447709,0.256865,0.652230,1.000000,0.047535,0.471516
455,0.000000,0.064122,0.000000,0.014517,0.019439,0.000000,0.000000,0.009703,0.009857,0.007493,...,0.032154,0.052342,0.000000,0.100241,0.107518,0.000000,0.000000,0.047535,1.000000,0.061242


In [28]:
# cosine similarity 값으로 
# 8조와 가장 비슷한 조를 찾기위한 과정
y_cosine = cosine_result_df[jo]
dic ={}
for i in range(len(y_cosine)):
    dic[str(posts_key[i])] = y_cosine[i]
dic = get_key(**dic)
#8조와 비슷한 조 찾기 위함
# euclidean distance 를 dic에 씌워 sorting 함(1부터 10위까지)
sorted(dic.items(), key=lambda x: x[1], reverse = True)[0:10]

[('OSH24', 1.0000000000000007),
 ('OSH26', 0.8062937734615139),
 ('OSH25', 0.7107142857142856),
 ('OSH35', 0.699653932312254),
 ('OSH49', 0.6932843600453533),
 ('OSH32', 0.6582108517973013),
 ('OSH47', 0.6471576481489799),
 ('OSH19', 0.6333941514110046),
 ('OSH4', 0.6177658256852798),
 ('OSH27', 0.6166548125935098)]

In [29]:
import openpyxl
# 워크북 열기
wb = openpyxl.load_workbook('count결과(C).xlsx')
# 시트 열기(활성화)
sheet = wb['안전보건법령 간']

# 제시된 틀 만들기
sheet['A1'] = '안전보건법령 간 count결과(C) 기준'
for i in range(10) :
    sheet.cell(row=2, column=i+2).value = i+1
for i in range(0,141) :
    s = "OSH"+str(i+1)
    sheet.cell(row=i+3, column=1).value = s

# 상위 유사조항 10개   
for j in range(0,141) :
    y_cosine = cosine_result_df[j]
    dic ={}
    for i in range(len(y_cosine)):
        dic[str(posts_key[i])] = y_cosine[i]
    dic = get_key(**dic)
    a = sorted(dic.items(), key=lambda x: x[1], reverse = True)[1:11]
    for i in range(10) :
        sheet.cell(row=j+3, column=i+2).value = str(a[i][0])
# 저장
wb.save('count결과(C).xlsx')